# Flag regions

AMRClaw and GeoClaw version 5.6.1 (and earlier) allow specifying rectangular refinement regions (see [www.clawpack.org/regions.html](http://www.clawpack.org/regions.html)) in `setrun.py` in the form of a list that is appended to `rundata.regiondata.regions`:

```
rundata.regiondata.regions.append([minlevel,maxlevel,t1,t2,x1,x2,y1,y2])
```
This is a region that is active from time `t1` to `t2` over the spatial extent `[x1,x2,y1,y2]`.

The modified version of AMRClaw/GeoClaw supports a new approach to specifying regions that I propose calling `flagregions`.  This will also support simple rectangles and so should ultimately replace `regions` in both AMRClaw and GeoClaw.  The name `flagregions` is more descriptive of the function served by these regions and may cause less confusion with other uses of the word "region" in various applications.

**NOTE:** The class `FlagRegion` discussed below is currently implemented in a separate module `data_FlagRegions.py` but should eventually be moved into `clawpack.amrclaw.data`.

The new way of specifying a flag region in `setrun.py` is to first define an object `flagregion` of class `FlagRegion`, set various attributes of this object (including `minlevel`, `maxlevel`, `t1`, `t2`, and a spatial extent), and then append this object to `rundata.flagregiondata.flagregions`.  Here is an example:

```
flagregion = data_FlagRegions.FlagRegion(num_dim=2)
flagregion.name = 'Region_AdmiraltyInlet'
flagregion.minlevel = 4
flagregion.maxlevel = 5
flagregion.t1 = 3600.
flagregion.t2 = 1e9
flagregion.spatial_region_type = 2  # Ruled Rectangle
flagregion.spatial_region_file = os.path.abspath('RuledRectangle_AdmiraltyInlet.data')
rundata.flagregiondata.flagregions.append(flagregion)
```

Note that the spatial extent in this case is specified by a path to a file that describes a Ruled Rectangle, in this case one that covers Admiralty Inlet as defined in one of the examples of  [RuledRectangles.ipynb](RuledRectangles.ipynb).

Simple rectangles (like the old "regions") can be specified via:

```
flagregion.spatial_region_type = 1  # Rectangle
flagregion.spatial_region = [x1,x2,y1,y2]  # the extent
```

For the time being, one can also specify regions the old way and can specify both "regions" and "flagregions" in the same `setrun.py`.  The former are written out to `regions.data` as always and the latter are written out to `flagregions.data`, and both sets of regions are read in by code in `region_module.f90` and used in `flagregions2.f90`.  The new versions of these files are in this repository in `new_fortran_amrclaw`.  


## Possible future enhancements

### Other attributes

In the future we may implement the ability to specify different tolerances or even different flagging methods in each region.  If so, other attributes such as `flagregion.tolerance` or `flagregion.method` could be easily added.

In [ ]:
from IPython.display import Image

## Example

The code in `examples/amrclaw_flagregions` gives an example based on the standard amrclaw advecting square example (see [plots in the gallery](http://www.clawpack.org/gallery/_static/amrclaw/examples/advection_2d_square/_plots/_PlotIndex.html).

This `setrun.py` file contains the following:

```

    # ---------------
    # NEW flagregions
    # ---------------

    # Eventually add this to amrclaw.data but for now in data_FlagRegions
    rundata.add_data(data_FlagRegions.FlagRegionData(num_dim=2), 'flagregiondata')
    flagregions = rundata.flagregiondata.flagregions  # initialized to []

    # now append as many flagregions as desired to this list:

    # The entire domain restricted to level 1 for illustration:
    # Note that this is a rectangle specified in the new way:
    # (other regions below will force/allow more refinement)
    flagregion = data_FlagRegions.FlagRegion(num_dim=2)
    flagregion.name = 'Region_domain'
    flagregion.minlevel = 1
    flagregion.maxlevel = 1
    flagregion.t1 = 0.
    flagregion.t2 = 1e9
    flagregion.spatial_region_type = 1  # Rectangle
    flagregion.spatial_region = [0.,1.,0.,1.]  # = [x1,x2,y1,y2]
    flagregions.append(flagregion)

    # A more general ruled rectangle:
    flagregion = data_FlagRegions.FlagRegion(num_dim=2)
    flagregion.name = 'Region_triangle'
    flagregion.minlevel = 2
    flagregion.maxlevel = 3
    flagregion.t1 = 0.
    flagregion.t2 = 1e9
    flagregion.spatial_region_type = 2  # Ruled Rectangle
    flagregion.spatial_region_file = \
            os.path.abspath('RuledRectangle_Triangle.data')
    flagregions.append(flagregion)

    # code to make RuledRectangle_Triangle.data:
    rr = region_tools.RuledRectangle()
    rr.method = 1 # piecewiselinear edges between s values
    rr.ixy = 'x'  # so s refers to x, lower & upper are limits in y
    rr.s = np.array([0.1, 0.8])
    rr.lower = np.array([0.2, 0.8])
    rr.upper = np.array([0.8, 0.8])
    rr.write('RuledRectangle_Triangle.data')
```

In this example, two flagregions are defined. The first is the full domain specified as a rectangle, with `maxlevel = 1` so that refinement is not allowed except within the second flagregion, which is a triangle defined as a ruled rectangle.  In this triangle, refinement to level 2 is forced and to level 3 is allowed.  

One additional change was to set
```
    amrdata.clustering_cutoff = 0.95
```
so that grids cannot extent too far out of the flagregion, for clarity in this simple example.

The initial data is a scalar field with value 1 inside a square and 0 outside, as shown in this contour plot:  

In [ ]:
Image('../examples/amrclaw_flagregions/contours_frame00.png', width=400)

The figure below shows the grid cells on levels 1 and 2 and only the patches on level 3, which is refined by an additional factor of 2.  The blue triangle is the flagregion.

In [ ]:
Image('../examples/amrclaw_flagregions/grids_frame00.png', width=400)

[Back to Index.ipynb](Index.ipynb)